## Insert and Retrieve 10,000 Numbers with Redis

### Requirements

1. Insert numbers 1 to 10,000 into Redis (Server A)
2. Retrieve and print them in reverse order from Redis Enterprise (Server B)

---

### Chosen Redis Data Type: `List`

We use a Redis **List** to store the sequence of numbers because:

- Redis Lists maintain insertion order
- We can append values efficiently using `RPUSH`

### Data Synchronization

We have configured **Server B (Redis Enterprise)** as a **replica** of **Server A**

This allows data written to Server A to automatically replicate to Server B in real time.


In [3]:
# Connect to Server A (Open Source Redis)
import redis

r1 = redis.Redis(
    host=os.environ['REDIS_SERVER1_IP'],
    port=6379,
    password=os.environ['REDIS_SERVER1_PASSWORD'],
    decode_responses=True
)

# Connect to Server B (Redis Enterprise)
r2 = redis.Redis(
    host=os.environ['REDIS_SERVER2_IP'],
    port=10878,
    decode_responses=True
)

## We use the pipeline feature in redis-py which lets batch multiple Redis commands so they are sent in one network request instead of one at a time. 

## It allows executing thousands of redis commands using the pipe.execute() command

## We use the `RPUSH` command to insert values at the tail of the list.
This ensures that the first value inserted (1) stays at index 0, and the last value (10,000) ends up at the last index.


In [4]:
r1.delete("numbers:list")  # Clear old data if present

pipe = r1.pipeline()
for i in range(1, 10001):
    pipe.rpush("numbers:list", i)
pipe.execute()

print("Inserted 10,000 numbers into Server A.")

Inserted 10,000 numbers into Server A.


In [6]:
values = r2.lrange('numbers:list', 0, -1)
print("Server 2 numbers:list length:", len(values))
print("First 5 values in reverse order:", list(reversed(values))[:5])

Server 2 numbers:list length: 10000
First 5 values in reverse order: ['10000', '9999', '9998', '9997', '9996']


In [7]:
print("Total elements in list:", r2.llen("numbers:list"))

Total elements in list: 10000


## Summary on inserting millions of values

Redis processes commands with a **single‐threaded** event loop, trading parallelism for simplicity and atomicity, yet achieves **sub‑millisecond latency** by keeping all data in memory. To ingest **hundreds of millions** of items at high throughput, we combine **parallelization** (multiple client threads + pipelining) and **sharding** (Redis Cluster) to fully utilize network, CPU, and memory bandwidth without introducing wait time. 
On the read side, we choose data structures (`LIST` with `LPUSH`/`LRANGE` or `SORTED SET` with `ZADD`/`ZREVRANGE`) that naturally return **newest‐first**, and we coordinate consumer clients to pull from each shard and merge in reverse chronological order.

---


## Leveraging Parallelization & Shards

### 1. Parallelization via Client‑Side Threads & Pipelines

Even though Redis is single‑threaded, we can **overlap** client CPU work (building commands) with server execution by using multiple application threads:

- **Threads** in your producer program (Python/Java/Go) each maintain their own Redis‐client connection.  
- Each thread **buffers** a batch (e.g. 5 000–50 000) of `LPUSH` or `ZADD` commands in a pipeline.  
- While Redis processes one batch, **other threads** prepare and send their next batches, keeping the server’s event loop fully saturated

### 2. Sharding with Redis Cluster

To go beyond one core:

1. **Redis Cluster** splits the keyspace into **16 384 hash slots**, assigning each slot to a different node (shard) 
2. The client library computes each key’s slot and routes commands to the correct node, so writes to `numbers:shard:3` go to Shard 3, `:shard:7` to Shard 7, etc.  
3. This multiplies throughput by the number of nodes—each running its own single‑threaded event loop on a separate core

---


## Sorted Sets option

You can use **Sorted Sets** to maintain strict insertion order by using each numeric ID as both the member and its score—so that
ZREVRANGE myzset 0 -1 automatically returns the highest (newest) IDs first. In a Redis Cluster, shard your data into keys like myzset:{0}, myzset:{1}, … to distribute across nodes, with the RedisCluster client handling routing of each ZADD. Producers and consumers each run in their own threads or processes with dedicated connections—producers pipeline batched ZADD calls to their shard to maximize throughput, while consumers page through ZREVRANGE on each shard and merge or process results in reverse chronological order. This pattern scales horizontally across cores and nodes and preserves strict ordering, at the cost of O(log N) write complexity and additional memory overhead for scores and indexes.


## AMR vs Elastic Cache
Azure Cache for Redis (AMR) runs the official Redis Enterprise engine, so you get built‑in modules like RediSearch, RedisJSON and RedisBloom. Amazon ElastiCache uses open‑source Redis (or AWS Valkey for optional multi‑threading) and does not include these enterprise modules

AMR supports active‑active geo‑replication for multi‑region read/write with conflict resolution, and offers an Enterprise Flash tier up to 4.5 TB on NVMe SSDs. ElastiCache only provides passive (read‑only) replicas in secondary regions and is strictly in‑memory.

Because of its module support, active‑active replication, and flash storage, AMR is ideal for advanced caching, AI/ML feature stores, and large‑scale apps. ElastiCache remains a solid, cost‑effective choice for basic caching, session storage, and general‑purpose Redis workloads.